In [1]:
import tensorflow as tf 
from tensorboard.plugins.hparams import api as hp

#### Import il dataset mnist

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4422102/4422102 [==============================] - 1s 0us/step


#### Creo i parametri di unita delle celle dei layers , quantitá di dropout, e ottimizzatori. E creo il file nella cartella di logs di TensoBoards

In [3]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

#### Addestro il modello con le varie combinazioni di parametri

In [4]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax),
    ])
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )

    model.fit(x_train, y_train, epochs=1)
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy

#### Creo una funzione che memorizza le combinazioni di parametri utilizzati

In [5]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

#### Eseguo gli addestramenti

In [6]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            run_name = "run-%d" % session_num
            print('--- Inizio test: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Inizio test: run-0
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}
313/313 [==============================] - 1s 2ms/step - loss: 0.5670 - accuracy: 0.7888
--- Inizio test: run-1
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'sgd'}
313/313 [==============================] - 1s 2ms/step - loss: 0.6900 - accuracy: 0.7690
--- Inizio test: run-2
{'num_units': 16, 'dropout': 0.5, 'optimizer': 'adam'}
313/313 [==============================] - 1s 2ms/step - loss: 0.6222 - accuracy: 0.7987
--- Inizio test: run-3
{'num_units': 16, 'dropout': 0.5, 'optimizer': 'sgd'}
313/313 [==============================] - 1s 2ms/step - loss: 0.8720 - accuracy: 0.7367
--- Inizio test: run-4
{'num_units': 32, 'dropout': 0.2, 'optimizer': 'adam'}
313/313 [==============================] - 1s 2ms/step - loss: 0.4568 - accuracy: 0.8387
--- Inizio test: run-5
{'num_units': 32, 'dropout': 0.2, 'optimizer': 'sgd'}
313/313 [==============================] - 1s 2ms/step - loss: 0.6404 - accuracy: 0.7738
-

eseguito lo script, in una shell avviare
Tensorboard con il seguente comando
nella stessa cartella in cui è salvato
questo jupyter notebook
```
(venv) C:\....\tensorboard --logdir logs/hparam_tuning